In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
dataset = pd.read_csv("Regularities_by_liaisons_Trains_France.csv")

In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7806 entries, 0 to 7805
Data columns (total 32 columns):
 #   Column                                                                                                               Non-Null Count  Dtype  
---  ------                                                                                                               --------------  -----  
 0   Year                                                                                                                 7806 non-null   int64  
 1   Month                                                                                                                7806 non-null   float64
 2   Departure station                                                                                                    7806 non-null   object 
 3   Arrival station                                                                                                      7806 non-null   object 
 4   Average trav

In [4]:
dataset.isnull().sum()

Year                                                                                                                      0
Month                                                                                                                     0
Departure station                                                                                                         0
Arrival station                                                                                                           0
Average travel time (min)                                                                                                63
Number of expected circulations                                                                                          63
Number of cancelled trains                                                                                                0
Number of late trains at departure                                                                                        0
Average 

In [5]:
data = dataset.drop(['Comment (optional) delays at departure','Comment (optional) delays on arrival'],axis=1)

In [6]:
data = data.dropna()

In [7]:
data.isnull().sum()

Year                                                                                                                   0
Month                                                                                                                  0
Departure station                                                                                                      0
Arrival station                                                                                                        0
Average travel time (min)                                                                                              0
Number of expected circulations                                                                                        0
Number of cancelled trains                                                                                             0
Number of late trains at departure                                                                                     0
Average delay of late departing 

In [8]:
categorical = ['Departure station','Arrival station']
final_data = pd.get_dummies(data, columns=categorical)

In [9]:
X = final_data.drop(['Delay due to external causes','Delay due to railway infrastructure','Delay due to traffic management',
                     'Delay due to rolling stock','Delay due to station management and reuse of material',
                     'Delay due to travellers taken into account','Period'],axis = 1)
Y = final_data[['Delay due to external causes','Delay due to railway infrastructure','Delay due to traffic management',
                     'Delay due to rolling stock','Delay due to station management and reuse of material',
                     'Delay due to travellers taken into account']]

In [10]:
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

In [11]:
# Standardize the input features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
# Reshape data for 1D CNN
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [13]:
# Define the CNN model with additional layers
model = keras.Sequential()

In [15]:
# Add Convolutional Layers
model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Flatten())

In [16]:
# Add Dense Layers
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))

In [17]:
# Define separate output layers for each target column
output_layer_names = Y.columns
output_layers = []

In [18]:
for column in output_layer_names:
    output_layers.append(layers.Dense(1, name=column))

In [19]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'], loss_weights=[1.0] * len(output_layers))

In [23]:
# Train the model
model.fit(X_train, [Y_train[column] for column in output_layer_names], epochs=100, batch_size=32, verbose=1)

Epoch 1/100
188/188 [==============================] - 4s 23ms/step - loss: 47.5692 - mse: 47.5692
Epoch 2/100
188/188 [==============================] - 4s 24ms/step - loss: 49.6512 - mse: 49.6512
Epoch 3/100
188/188 [==============================] - 5s 28ms/step - loss: 48.0054 - mse: 48.0054
Epoch 4/100
188/188 [==============================] - 8s 42ms/step - loss: 47.1086 - mse: 47.1086
Epoch 5/100
188/188 [==============================] - 11s 58ms/step - loss: 47.0646 - mse: 47.0646
Epoch 6/100
188/188 [==============================] - 10s 53ms/step - loss: 46.9735 - mse: 46.9735
Epoch 7/100
188/188 [==============================] - 10s 51ms/step - loss: 47.0838 - mse: 47.0838
Epoch 8/100
188/188 [==============================] - 8s 40ms/step - loss: 47.1529 - mse: 47.1529
Epoch 9/100
188/188 [==============================] - 9s 50ms/step - loss: 46.8957 - mse: 46.8957
Epoch 10/100
188/188 [==============================] - 10s 51ms/step - loss: 47.4308 - mse: 47.4308
Epoch

188/188 [==============================] - 4s 23ms/step - loss: 46.8565 - mse: 46.8565
Epoch 83/100
188/188 [==============================] - 5s 24ms/step - loss: 46.6719 - mse: 46.6719
Epoch 84/100
188/188 [==============================] - 4s 23ms/step - loss: 46.6949 - mse: 46.6949
Epoch 85/100
188/188 [==============================] - 4s 23ms/step - loss: 46.6509 - mse: 46.6509
Epoch 86/100
188/188 [==============================] - 4s 23ms/step - loss: 46.6811 - mse: 46.6811
Epoch 87/100
188/188 [==============================] - 5s 24ms/step - loss: 46.6937 - mse: 46.6937
Epoch 88/100
188/188 [==============================] - 4s 24ms/step - loss: 46.6500 - mse: 46.6500
Epoch 89/100
188/188 [==============================] - 4s 22ms/step - loss: 46.7116 - mse: 46.7116
Epoch 90/100
188/188 [==============================] - 4s 23ms/step - loss: 46.8531 - mse: 46.8531
Epoch 91/100
188/188 [==============================] - 4s 22ms/step - loss: 46.7997 - mse: 46.7997
Epoch 92/100


In [24]:
# Evaluate the model using MSE
Y_pred = model.predict(X_test)

# Calculate the mean squared error for each target column
mse = [mean_squared_error(Y_test[column], Y_pred[:, i]) for i, column in enumerate(output_layer_names)]


47/47 [==============================] - 0s 9ms/step


In [25]:
# Print the mean squared error for each target column
for i, column in enumerate(output_layer_names):
    print(f"Mean Squared Error for {column}: {mse[i]}")

Mean Squared Error for Delay due to external causes: 0.5578336794482416
Mean Squared Error for Delay due to railway infrastructure: 652.7434515875493
Mean Squared Error for Delay due to traffic management: 683.2585369464655
Mean Squared Error for Delay due to rolling stock: 533.0799331121325
Mean Squared Error for Delay due to station management and reuse of material: 712.2454853641736
Mean Squared Error for Delay due to travellers taken into account: 802.4657486261025
